## IMPORTACIÓN DEL DATASET

In [218]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargo el archivo CSV
items_titles   = pd.read_csv(r'C:\Users\sfaija\Challenge_Data_Science\items_titles.csv')
items_titles_test   = pd.read_csv(r'C:\Users\sfaija\Challenge_Data_Science\items_titles_test.csv')

# Muestro los primeros registros para tener una idea de la estructura del archivo
items_titles.head()

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


In [219]:
print("Cantidad de",items_titles.count())
print("Cantidad de test",items_titles_test.count())

Cantidad de ITE_ITEM_TITLE    30000
dtype: int64
Cantidad de test ITE_ITEM_TITLE    10000
dtype: int64


Cuento con 30.000 titulos que debo agrupar en las familias que se encuentran en items_titles_test

### Modelo de Vecinos Más Cercanos (Nearest Neighbors) para Similitud de Títulos

A Continuación implementaré un modelo de Vecinos Más Cercanos para identificar los títulos más similares en el conjunto de datos principal para cada título en el conjunto de datos de prueba. Utilizaré la métrica del coseno para medir cuán similares son dos títulos basados en su representación TF-IDF (permite convertir texto sin estructura en un vector numérico).

Estos vectores luego pueden compararse para determinar la similitud entre diferentes textos. 
##### Cuanto más alto sea el valor de la similitud del coseno, más similares serán los dos títulos.

In [221]:
from sklearn.neighbors import NearestNeighbors

# Defini que quiweo recuperar los 10 títulos más similares (vecinos) para cada título en el conjunto de datos de prueba.
num_neighbors = 10

# Entreno el modelo de Vecinos Más Cercanos usando los títulos del conjunto de datos principal.
nn_model = NearestNeighbors(metric='cosine').fit(tfidf_main)

# Busco los títulos más similares en el conjunto de datos principal para cada título en el conjunto de datos de prueba.
distances, indices = nn_model.kneighbors(tfidf_test, n_neighbors=num_neighbors)

# Creo resultado en DATAFRAME
results = {
    "ite_item_title_1": [],
    "ite_item_title_2": [],
    "score_similitud": []
}

for idx, title in enumerate(items_titles_test["ITE_ITEM_TITLE"]):
    for jdx, main_idx in enumerate(indices[idx]):
        results["ite_item_title_1"].append(title)
        results["ite_item_title_2"].append(items_titles.iloc[main_idx]["ITE_ITEM_TITLE"])
        results["score_similitud"].append(1 - distances[idx][jdx])  # Convert distance to similarity


similarity_df_sklearn = pd.DataFrame(results)


similarity_df_sklearn = similarity_df_sklearn.sort_values(by="score_similitud", ascending=False)

similarity_df_sklearn.head()


,ite_item_title_1,ite_item_title_2,score_similitud
89660,Tênis Infantil Feminino Molekinha Led Lançamen...,Tênis Feminino Molekinha Led Infantil Lançamen...,1.0
29160,Tênis Feminino / Masculino Preto Crepe Neutral Mm,Tênis Feminino / Masculino Preto Crepe Neutral Mm,1.0
93270,Sapatênis Masculino Pegada - 114856,Sapatênis Pegada Masculino 517153,1.0
93271,Sapatênis Masculino Pegada - 114856,Sapatênis Masculino Pegada 17120105,1.0
93272,Sapatênis Masculino Pegada - 114856,Sapatênis Pegada Masculino 118402-01,1.0


In [222]:
# Para mostrar los títulos con mayor similitud

top_similar_titles = similarity_df_sklearn.head(5)
top_similar_titles

,ite_item_title_1,ite_item_title_2,score_similitud
89660,Tênis Infantil Feminino Molekinha Led Lançamen...,Tênis Feminino Molekinha Led Infantil Lançamen...,1.0
29160,Tênis Feminino / Masculino Preto Crepe Neutral Mm,Tênis Feminino / Masculino Preto Crepe Neutral Mm,1.0
93270,Sapatênis Masculino Pegada - 114856,Sapatênis Pegada Masculino 517153,1.0
93271,Sapatênis Masculino Pegada - 114856,Sapatênis Masculino Pegada 17120105,1.0
93272,Sapatênis Masculino Pegada - 114856,Sapatênis Pegada Masculino 118402-01,1.0


In [223]:
# Para mostrar los títulos con baja similitud

bottom_similar_titles = similarity_df_sklearn.tail(5)
bottom_similar_titles

,ite_item_title_1,ite_item_title_2,score_similitud
37403,J1 Seafoam,Tênis Menino Infantil Slip On Conforto Molekin...,0.0
37404,J1 Seafoam,Tenis Mizuno Grafite Preto Laranja Masculino W...,0.0
37405,J1 Seafoam,Tênis Infantil Bibi Masculino Fisioflex 3.0 Cinza,0.0
37406,J1 Seafoam,Tênis adidas Bd8055,0.0
39700,Piso,Tenis Feminino Beira Rio Napa Turim Rosa Trato...,0.0


La columna ite_item_title_1 contiene el título del conjunto de datos de prueba (items_titles_test), y la columna ite_item_title_2 contiene el título del conjunto de datos principal (items_titles) que fue encontrado como similar.

Ya a simple vista podemos ver en el ejemplo "Sapatênis Masculino Pegada - 114856	" con Sapatênis Pegada Masculino 118402-01	que el score similitud nos da 1 porque sus representaciones vectoriales (en este caso, TF-IDF) tienen un ángulo de coseno de 1, lo que indica que son muy similares en términos del contenido de palabras. Pero no es exactamente igual el titutlo, entonces hare una siguiente revision y emplearé otras tecnicas:

###### 1. Ponderacion de caracteres: puedo asignar un peso mayor a los dígitos para reflejar su importancia.
Voy a modificar el TfidfVectorizer para que asigne un peso mayor a los números: 

Defino el tokenizador personalizado para dar más peso a los números (como se hizo anteriormente).
Crear un nuevo TfidfVectorizer con el tokenizador personalizado y ajustar/transformar tus datos.
Luego voy a entrenar el modelo NearestNeighbors con los vectores TF-IDF personalizados y encontrar los títulos más similares como lo hice anteriormente.

In [224]:
import re  ## Es el módulo de Python para trabajar con expresiones regulares. Se usa para operaciones de búsqueda y manipulación de texto basadas en patrones.
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS  ## Importo una lista predefinida de stopwords en inglés proporcionada por scikit-learn por si la llego a utilizar.
import nltk  ## Natural Language Toolkit, una biblioteca para procesamiento de lenguaje natural.
from nltk.corpus import stopwords  ## Importo la función que me permite acceder a las stopwords.

# Descargo la lista de stopwords
nltk.download('stopwords')

# Obtengo las stopwords en portugués
portuguese_stopwords = set(stopwords.words('portuguese'))

## Convierto todo el texto a letras minúsculas para asegurarme de que las palabras sean tratadas de manera uniforme, independientemente de cómo estén escritas originalmente.
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Eliminar caracteres que no sean alfanuméricos 
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenizar el texto y filtrar stopwords en inglés y portugués
    tokens = text.split()
    tokens = [token for token in tokens if token not in portuguese_stopwords and token not in ENGLISH_STOP_WORDS]
    
    return ' '.join(tokens)

# Aplico el procesamiento para cada titulo del dataset
items_titles['ITE_ITEM_TITLE'] = items_titles['ITE_ITEM_TITLE'].apply(preprocess_text)
items_titles_test['ITE_ITEM_TITLE'] = items_titles_test['ITE_ITEM_TITLE'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sfaija\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Ponderación Diferente


In [225]:
from sklearn.feature_extraction.text import TfidfVectorizer ##herramienta de scikit-learn que convierte una colección de documentos de texto en una matriz de características TF-IDF.
from sklearn.neighbors import NearestNeighbors ##modelo de scikit-learn que encuentra los puntos más cercanos en un conjunto de puntos.


# Ponderación Diferente

# Definición de un Tokenizador Personalizado (custom_tokenizer):Esta función toma un texto como entrada y lo divide en tokens (palabras) usando espacios.Si un token tiene un número (es decir, es numérico o tiene dígitos), multiplica ese token por 5. Esto le da más peso a los números, lo que significa que los números serán más "importantes" o "relevantes" en la representación TF-IDF.

def custom_tokenizer(text):
    tokens = text.split()
    tokens = [token if not bool(re.search(r'\d', token)) else token*5 for token in tokens]
    return tokens

# creo un TfidfVectorizer que utiliza el tokenizador personalizado definido anteriormente. Concateno el conjunto principal de títulos con el conjunto de prueba y se ajusta y transforma utilizando el vectorizador personalizado. El resultado es una matriz TF-IDF que representa todos los títulos.
custom_vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)

# Entrenamiento del Modelo de Vecinos Más Cercanos:
all_titles_custom = pd.concat([items_titles["ITE_ITEM_TITLE"], items_titles_test["ITE_ITEM_TITLE"]])
tfidf_matrix_custom = custom_vectorizer.fit_transform(all_titles_custom)

# Búsqueda de Títulos Similares:
tfidf_main_custom = tfidf_matrix_custom[:len(items_titles)]
tfidf_test_custom = tfidf_matrix_custom[len(items_titles):]

# Conversión y Ordenación de Resultados:
nn_model_custom = NearestNeighbors(metric='cosine').fit(tfidf_main_custom)

distances_custom, indices_custom = nn_model_custom.kneighbors(tfidf_test_custom, n_neighbors=3)

results_custom = {
    "ite_item_title_1": [],
    "ite_item_title_2": [],
    "score_similitud": []
}

for idx, titulo in enumerate(items_titles_test["ITE_ITEM_TITLE"]):
    for jdx, main_idx in enumerate(indices_custom[idx]):
        results_custom["ite_item_title_1"].append(titulo)
        results_custom["ite_item_title_2"].append(items_titles.iloc[main_idx]["ITE_ITEM_TITLE"])
        results_custom["score_similitud"].append(1 - distances_custom[idx][jdx])  # Convert distance to similarity

# convierto en DataFrame 
similarity_df_custom = pd.DataFrame(results_custom)
similarity_df_custom_sorted = similarity_df_custom.sort_values(by="score_similitud", ascending=False)

similarity_df_custom_sorted.head()



C:\Users\sfaija\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,ite_item_title_1,ite_item_title_2,score_similitud
1668,bicicleta aro 29 dropp rs1 pro 27v hidra trava...,bicicleta aro 29 dropp rs1 pro 27v hidra trava...,1.0
5346,sapatênis feminino casual detalhe onça cadarço,sapatênis feminino casual detalhe onça cadarço,1.0
23574,tenis masculino calçado masculino confortavel ...,calçado masculino tenis masculino confortavel ...,1.0
27879,tênis asics kabuki feminino caminhada,tênis asics kabuki feminino caminhada,1.0
12369,tênis feminino chunky ramarim sneaker tratorad...,tênis feminino chunky ramarim sneaker tratorad...,1.0


Defino la funcion ajustar_similitud para ajustar las puntuaciones de similitud basadas en la presencia de números en los títulos. Esto es importante porque si dos productos tienen números diferentes en sus títulos, aunque sus títulos sean textualmente similares, podrían ser conceptualmente diferentes.

In [226]:
def ajustar_similitud(fila):
    # Extraigo números de los títulos
    numeros_titulo_1 = set(re.findall(r'\d+', fila["ite_item_title_1"]))
    numeros_titulo_2 = set(re.findall(r'\d+', fila["ite_item_title_2"]))
    
    # Si los números son diferentes, reduzcoo la puntuación de similitud
    if numeros_titulo_1 != numeros_titulo_2:
        return fila["score_similitud"] * 0.75  # Reducción del 25%
    else:
        return fila["score_similitud"]

# Aplico la función para ajustar las puntuaciones de similitud
similarity_df_custom["score_similitud"] = similarity_df_custom.apply(ajustar_similitud, axis=1)

similarity_df_custom.head()


,ite_item_title_1,ite_item_title_2,score_similitud
0,tênis olympikus esporte valente masculino kids,tênis infantil olympikus valente kids masculino,0.858660
1,tênis olympikus esporte valente masculino kids,tênis olympikus valente menino cadarço,0.620575
2,tênis olympikus esporte valente masculino kids,tênis esporte masculino,0.514669
3,bicicleta barra forte samy c 6 marchas cubo c ...,bicicleta barra forte aro 24,0.317830
4,bicicleta barra forte samy c 6 marchas cubo c ...,bicicleta aro 26 barra forte fem new bege,0.263160


In [227]:
# Ordeno el DataFrame
similarity_df_custom_sorted = similarity_df_custom.sort_values(by=['ite_item_title_1', 'score_similitud'], ascending=[True, False])

# Muestro las primeras filas del DataFrame ordenado
similarity_df_custom_sorted.head(20)


,ite_item_title_1,ite_item_title_2,score_similitud
7290,02 tênis feminino academia olimp olymp willian...,02 tênis sapatilha feminina olimp olymp willia...,0.761284
7291,02 tênis feminino academia olimp olymp willian...,kit 3 pares tênis olymp olimp willians feminin...,0.542850
7292,02 tênis feminino academia olimp olymp willian...,kit 2 tênis feminino academia olimp olymp cami...,0.478150
20055,04 kits friso refletivo bike bicicleta aero ar...,bicicleta mountain bike aro 24 aero feminina rosa,0.294562
20056,04 kits friso refletivo bike bicicleta aero ar...,bike aero,0.275821
20057,04 kits friso refletivo bike bicicleta aero ar...,bike aro 700 speed,0.270852
27258,1 par sapatenis casual masculino couro legitim...,kit 1 par sapatenis,0.672832
27259,1 par sapatenis casual masculino couro legitim...,tenis sapatenis casual masculino couro legitimo,0.495337
27260,1 par sapatenis casual masculino couro legitim...,sapatenis couro legitimo lona masculino promoção,0.466576
25248,1053221 tênis bibi evolution tecido naval pret...,tênis infantil bibi evolution masculino azul n...,0.816823


In [228]:
similarity_df_custom_sorted.to_csv("similarity_df_custom.csv", index=False)


### EVALUACION DE EFECTIVIDAD DEL MODELO:
Como no cuento con una base para testear si fue correctamente asignado la similitud, una alternativa es hacer un analisis cualitativo y ver casos de ejemplos.

Aunque no tengo datos etiquetados, se podría crear un pequeño conjunto de datos etiquetado. Por ejemplo, seleccionar aleatoriamente algunos títulos y etiquetar manualmente los títulos más similares. Luego, podemos usar este conjunto de datos para calcular métricas como precisión, recall y F1-score.

Aparte de Vecinos Más Cercanos, hay otros modelos y técnicas que se pueden utilizar, como modelos basados en embeddings  o incluso soluciones basadas en deep learning.